En este archivo se preparan datasets especificos para cada endpoint de nuestra api. Realizaremos aqui calculos y operaciones pesadas que puedan ralentizar nuestras consultas, y tambien nos ahorramos cargar datasets completos en nuestro servidor lo que podria llevar a una sobrecarga de memoria.

In [1]:
import pandas as pd


Importamos los datasets generados en nuestras anteriores ETL

In [2]:
df_games_complete=pd.read_parquet('processed_data/games.parquet')
df_items_complete=pd.read_parquet('processed_data/items.parquet')
df_reviews_complete=pd.read_parquet('processed_data/reviews.parquet')

Preparacion de dataset para la funcion PlayTimeGenre

In [3]:
#Se crean dataframes solo con las columnas que vamos a utilizar
df_games=df_games_complete[['genres','release_year','id']]
df_items=df_items_complete[['item_id','playtime_forever']]

In [5]:
#Se realiza un explode de los generos
df_games=df_games.explode('genres',ignore_index=False,)

#Se hace un join entre df_games y df_items usando 'id' y 'item_id'
df_games['id'] = df_games['id'].astype(int)
df_items['item_id'] = df_items['item_id'].astype(int)
df_joined = pd.merge(df_games, df_items, left_on='id', right_on='item_id', how='inner')

#Se agrupa por género y año, sumando las horas jugadas
df_grouped = df_joined.groupby(['genres', 'release_year'])['playtime_forever'].sum().reset_index()
#Se encuentra el índice del máximo de horas jugadas para cada género
idx = df_grouped.groupby('genres')['playtime_forever'].idxmax()
#Se filtra df_grouped usando los índices encontrados
df_max_year_per_genre = df_grouped.loc[idx]


C:\Users\PC\AppData\Local\Temp\ipykernel_5432\2050125367.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items['item_id'] = df_items['item_id'].astype(int)


In [ ]:
#Se crea un dataframe solo con las columnas necesarias para nuestra consulta y se guarda para su posterior uso
df_function_PlayTimeGenre=df_max_year_per_genre[['genres','release_year']]
df_function_PlayTimeGenre.to_parquet('processed_data/play_time_genre.parquet')


Preparacion de dataset para la funcion UserForGenre

In [7]:
#Se crean dataframes solo con las columnas que vamos a utilizar
df_games=df_games_complete[['genres','release_year','id']]
df_items=df_items_complete[['user_id','item_id','playtime_forever']]

In [8]:
#Se realiza un explode de los generos
df_games=df_games.explode('genres',ignore_index=False,)

#Se hace un join entre df_games y df_items usando 'id' y 'item_id'
df_games['id'] = df_games['id'].astype(int)
df_items['item_id'] = df_items['item_id'].astype(int)
df_joined = pd.merge(df_games, df_items, left_on='id', right_on='item_id', how='inner')

#Se agrupa por género, usuario y año, sumando las horas jugadas
df_grouped = df_joined.groupby(['genres','user_id', 'release_year'])['playtime_forever'].sum().reset_index()

#Se calcula el total de horas jugadas por usuario y genero
df_total_hours = df_grouped.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()

#Se busca al usuario con mas horas jugadas por genero
idx = df_total_hours.groupby('genres')['playtime_forever'].idxmax()
df_user_max_hours = df_total_hours.loc[idx]

#Se filtra el dataframe completo para obtener los registros del usuario con mas horas jugadas para cada genero
df_max_user_per_genre = pd.merge(df_joined, df_user_max_hours[['genres', 'user_id']], on=['genres', 'user_id'], how='inner')

#Se agrupa por género, usuario y año el dataframe filtrado , sumando las horas jugadas
df_max_user_per_genre=df_max_user_per_genre.groupby(['genres','user_id','release_year'])['playtime_forever'].sum().reset_index()


In [ ]:
#Se crea un dataframe solo con las columnas necesarias para nuestra consulta y se guarda para su posterior uso
df_function_UserForGenre=df_max_user_per_genre[['genres','release_year','user_id','playtime_forever']]
df_function_UserForGenre.to_parquet('processed_data/user_for_genre.parquet')

Preparacion de dataset games para utilizar en los endpoints UsersWorstDeveloper, sentiment_analysis y recomendacion_juego

In [10]:
#Se crea el dataframe solo con las columnas que se van a utilizar y se guarda en formato parquet para su posterior uso.
df_games=df_games_complete[['id','app_name','developer']]
df_games.to_parquet('processed_data/games_endpoints.parquet')